In [1]:
from pymongo import MongoClient
from tensorflow.python.summary import event_accumulator as ea
import matplotlib
matplotlib.use('Qt4Agg')
import matplotlib.pyplot as plt
import numpy as np

client = MongoClient("mongodb://localhost:27017")

In [2]:
from bson.objectid import ObjectId

In [3]:


#cusor = client['sacred']['runs'].find({'config.lr' : {'$lt': 0.1}})
cusor = client['cifar_ten']['runs'].find({})


In [4]:
tb_dir = cusor[4]['config']['tensorboard_dir']

#print client['sacred'].collection_names()
print cusor[4]['experiment']['name']

#print client['sacred']['fs.files'].find({'_id' : ObjectId('5974cf42b7d25c1df647caa9')})[0]
print cusor[4]['config']

cifar_sgd_adjust
{u'state_of_the_art': True, u'per_variable': False, u'iters_per_adjust': 250, u'num_residual_units': 4, u'relu_leakiness': 0.1, u'batch_size': 100, u'n_epochs': 100, u'base_optimizer': u'SGD', u'seed': 402831889, u'lr': 0.1, u'beta2': 0.999, u'beta1': 0.9, u'rho': 0.95, u'use_bottleneck': False, u'tensorboard_dir': u'/home/shai/tensorflow/generated_data/167', u'weight_decay_rate': 0.0002}


In [5]:

acc = ea.EventAccumulator(tb_dir)
acc.Reload()

INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/167/events.out.tfevents.1500847782.gpu-plx01


In [6]:
print acc.Tags()['scalars']
print tb_dir

[u'train_loss', u'base_optimizer/weights_norm_during_sgd', u'train_accuracy', u'base_optimizer/learning_rate', u'base_optimizer/grad_norm_norm_during_sgd', u'test_accuracy', u'test_loss']
/home/shai/tensorflow/generated_data/167


In [7]:
#

In [8]:

#acc.Scalars('train_loss')

In [3]:

class Plotter:
    accs_cache = {}

    @staticmethod
    def get_accs(client, databases, query, asked_labels):
        tb_dirs = []
        labels = []
        batch_sizes = []
        configs = []
        print 'databases = ' + str(databases)
        for database in databases:
            cusor = client[database]['runs'].find(query)
            print 'cusor = ' + str(cusor)
            
            for x in cusor:
                print 'experiment name: ' + x['experiment']['name']
                tb_dirs.append(x['config']['tensorboard_dir'])
                labels.append([])
                batch_sizes.append(x['config']['batch_size'])
                configs.append(x['config'])
                for asked_label in asked_labels:
                    try:
                        labels[-1].append(asked_label + '=' + str(x['config'][asked_label]))
                    except KeyError:
                        labels[-1].append(asked_label + '=None')
                

        accs = []
        for d in tb_dirs:
            if d not in Plotter.accs_cache:
                Plotter.accs_cache[d] = ea.EventAccumulator(d)
                Plotter.accs_cache[d].Reload()
            accs.append(Plotter.accs_cache[d])
        
        
        l = zip(labels, batch_sizes, accs, configs)
        l.sort()
        labels, batch_sizes, accs, configs = zip(*l)

        return accs, labels, batch_sizes, configs
    
    @staticmethod
    def get_plot_time_to_accuracy(acc, batch_size, acc_labels):
        target_accs = []
        first_biggers = []
        for target_acc in np.linspace(0.2, 1, 10000):
            try:
                first_biggers.append((s.step/float(batch_size) for s in acc.Scalars('test_accuracy') if s.value > target_acc).next())
                target_accs.append(target_acc)
            except StopIteration:
                pass

        return target_accs, first_biggers
        
    @staticmethod
    def _plot_speedup_to_accuracy(ref_acc, ref_batch_size, acc, batch_size, acc_labels):
        target_accs, first_biggers = Plotter.get_plot_time_to_accuracy(acc, batch_size, acc_labels)
        ref_target_accs, ref_first_biggers = Plotter.get_plot_time_to_accuracy(ref_acc, ref_batch_size, acc_labels)

        x = np.array(target_accs, dtype=np.float32)
        y = np.array([s for s in first_biggers], dtype=np.float32)
        
        ref_y = np.array([s for s in ref_first_biggers], dtype=np.float32)
        
        #resize y_ref:
        ref_y = np.resize(ref_y, y.shape)
        
        plt.plot(target_accs, ref_y/y, label=str(acc_labels))
        
    @staticmethod
    def plot_speedup_to_accuracy(client, databases, ref_query, query, tag, asked_labels):
        accs, labels, batch_sizes, configs = Plotter.get_accs(client, databases, query, asked_labels)
        ref_accs, ref_labels, ref_batch_sizes, ref_configs = Plotter.get_accs(client, databases, ref_query, asked_labels)
        
        #plot
        fig = plt.figure(figsize=(10, 8))
        for acc, acc_labels, batch_size in zip(accs, labels, batch_sizes):
            Plotter._plot_speedup_to_accuracy(ref_accs[0], ref_batch_sizes[0], acc, batch_size, acc_labels)
            
    @staticmethod
    def _plot_time_to_accuracy(acc, batch_size, acc_labels):
        target_accs, first_biggers = Plotter.get_plot_time_to_accuracy(acc, batch_size, acc_labels)
        plt.plot(target_accs, [s for s in first_biggers], label=str(acc_labels))
    
    @staticmethod
    def plot_time_to_accuracy(client, databases, query, tag, asked_labels):
        accs, labels, batch_sizes, configs = Plotter.get_accs(client, databases, query, asked_labels)
        #plot
        fig = plt.figure(figsize=(10, 8))
        for acc, acc_labels, batch_size in zip(accs, labels, batch_sizes):
            Plotter._plot_time_to_accuracy(acc, batch_size, acc_labels)
 
    
    @staticmethod
    def plot(client, databases, query, tag, asked_labels, scale_by_batch_size=None, create_figure=True):
        accs, labels, batch_sizes, configs = Plotter.get_accs(client, databases, query, asked_labels)
        #plot
#         print 'accs = ' + str(accs)
#         print 'labels = ' + str(labels)
#         print 'batch_sizes = ' + str(batch_sizes)
#         print 'zip = ' + str(zip(accs, labels, batch_sizes))
        if create_figure:
            fig = plt.figure(figsize=(10, 8))
        for acc, acc_labels, batch_size in zip(accs, labels, batch_sizes):
            scalar = acc.Scalars(tag)
            #print 'scalar = ' + str(scalar)
            if scale_by_batch_size == None:
                plt.plot([s.step for s in scalar], [s.value for s in scalar], label=str(acc_labels))
            else:
                plt.plot([s.step/float(scale_by_batch_size(batch_size)) for s in scalar], 
                         [s.value for s in scalar], label=str(acc_labels))
            
    @staticmethod
    def plot_max(client, databases, query, tag, asked_labels, scale_by_batch_size=None):
        accs, labels, batch_sizes, configs = Plotter.get_accs(client, databases, query, asked_labels)
        #plot
        fig = plt.figure(figsize=(10, 8))
        for acc, acc_labels, batch_size in zip(accs, labels, batch_sizes):
            scalar = acc.Scalars(tag)
            maxes = [max([s.value for s in scalar[:i+1]] ) for i in range(len(scalar))]
            
            print 'max = ' + str(max([s.value for s in scalar] ))
            if scale_by_batch_size == None:
                plt.plot([s.step for s in scalar], maxes, label=str(acc_labels))
            else:
                plt.plot([s.step/float(scale_by_batch_size(batch_size)) for s in scalar], maxes, label=str(acc_labels))
                
    @staticmethod
    def plot_min(client, databases, query, tag, asked_labels, scale_by_batch_size=None):
        accs, labels, batch_sizes, configs = Plotter.get_accs(client, databases, query, asked_labels)
        #plot
        fig = plt.figure(figsize=(10, 8))
        for acc, acc_labels, batch_size in zip(accs, labels, batch_sizes):
            scalar = acc.Scalars(tag)
            maxes = [min([s.value for s in scalar[:i+1]] ) for i in range(len(scalar))]
            if scale_by_batch_size == None:
                plt.plot([s.step for s in scalar], maxes, label=str(acc_labels))
            else:
                plt.plot([s.step/float(scale_by_batch_size(batch_size)) for s in scalar], maxes, label=str(acc_labels))

# Results – MNIST on CNN

In [393]:
fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(221)
plt.title('Momentum Test Loss')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1673'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1661'}] }
Plotter.plot(client, ['minst_sgd_adjust_db'], query, 'test_loss', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Loss')
plt.legend()
plt.yscale('log')
plt.grid(True)



ax = fig.add_subplot(222)
plt.title('Nestrov Test Loss')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1677'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1669'}] }
Plotter.plot(client, ['minst_sgd_adjust_db'], query, 'test_loss', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Loss')
plt.legend()
plt.yscale('log')
plt.grid(True)



ax = fig.add_subplot(223)
plt.title('SGD Test Loss')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1675'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1667'}] }
Plotter.plot(client, ['minst_sgd_adjust_db'], query, 'test_loss', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Loss')
plt.legend()
plt.yscale('log')
plt.grid(True)


#ANGLE (for SGD)
ax = fig.add_subplot(224)
plt.title('Angle between recent trajectories')
query = {'$or': [
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1675'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1730'}] }
#Plotter.accs_cache = {}
#accs = Plotter.get_accs(client, ['minst_sgd_adjust_db'], query, ['lr', 'disable_lr_change'])

Plotter.plot(client, ['minst_sgd_adjust_db'], query, 'base_optimizer/angle__u__v_', 
             ['lr', 'disable_lr_change'], scale_by_batch_size=None, create_figure=False)

plt.title('Angle between recent trajectories')
plt.xlabel('1/2 epoch')
plt.ylabel('cos(angle)')

#plt.yscale('log')
plt.legend()
plt.grid(True)
plt.show()



plt.show()


databases = ['minst_sgd_adjust_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b47582b50>
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
databases = ['minst_sgd_adjust_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b4691a690>
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
databases = ['minst_sgd_adjust_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b46a3ee90>
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
databases = ['minst_sgd_adjust_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b47d9c9d0>
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust


# Cifar10 - w28-10 Wide Residual Network

In [341]:
fig = plt.figure(figsize=(10, 8))

ax = fig.add_subplot(221)
plt.title('Momentum Test Accuracy')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1638'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1639'}] }
Plotter.plot(client, ['cifar_ten'], query, 'test_accuracy', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Accuracy')
plt.legend()
plt.grid(True)



ax = fig.add_subplot(222)
plt.title('Nestrov Test Accuracy')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1637'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1643'}] }
Plotter.plot(client, ['cifar_ten'], query, 'test_accuracy', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Accuracy')
plt.legend()
plt.grid(True)



ax = fig.add_subplot(223)
plt.title('Adagrad Test Accuracy')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1644'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1645'}] }
Plotter.plot(client, ['cifar_ten'], query, 'test_accuracy', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Accuracy')
plt.legend()
plt.grid(True)



ax = fig.add_subplot(224)
plt.title('SGD Test Accuracy')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/166'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/167'}] }
Plotter.plot(client, ['cifar_ten'], query, 'test_accuracy', 
             ['lr', 'iters_per_adjust'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Accuracy')
plt.legend()
plt.grid(True)



plt.show()




databases = ['cifar_ten']
cusor = <pymongo.cursor.Cursor object at 0x7f1b46d51a10>
experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1638/events.out.tfevents.1506348358.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1639/events.out.tfevents.1506348410.gpu-plx01
databases = ['cifar_ten']
cusor = <pymongo.cursor.Cursor object at 0x7f1b4e2bf590>
experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1637/events.out.tfevents.1506348337.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1643/events.out.tfevents.1506348664.gpu-plx03
databases = ['cifar_ten']
cusor = <pymongo.cursor.Cursor object at 0x7f1b6b2e5390>
experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_dat

In [377]:
fig = plt.figure(figsize=(10, 8))

plt.title('SGD Test Accuracy')
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/167'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1723'}] }
Plotter.plot(client, ['cifar_ten'], query, 'test_accuracy', 
             ['lr', 'break_lr_hand_tune'], scale_by_batch_size=None, create_figure=False)
plt.xlabel('epoch')
plt.ylabel('Test Accuracy')
plt.legend()
plt.grid(True)
plt.show()

databases = ['cifar_ten']
cusor = <pymongo.cursor.Cursor object at 0x7f1b3777f110>
experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust


# The angle between recent trajectories

In [319]:
#plt.plot([s.value for s in accs[0][0].Scalars('base_optimizer/angle__u__v_')])
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1650'},
   # {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1651'},
   # {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1652'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1672'}] }
#Plotter.accs_cache = {}
#accs = Plotter.get_accs(client, ['minst_sgd_adjust_db'], query, ['lr', 'disable_lr_change'])

Plotter.plot(client, ['minst_sgd_adjust_db'], query, 'base_optimizer/angle__u__v_', 
             ['lr', 'disable_lr_change'], scale_by_batch_size=None)

plt.title('Angle between recent trajectories')
plt.xlabel('1/2 epoch')
plt.ylabel('cos(angle)')

#plt.yscale('log')
plt.legend()
plt.grid(True)
plt.show()

databases = ['minst_sgd_adjust_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b4538ed10>
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust


# The short blanket - tighten it up

In [410]:
# Mnist CNN with weight decay
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1724'}] }


#Plotter.accs_cache = {}
accs, _, _, configs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
print configs

Plotter.plot(client, ['minst_seboost_db'], query, 
             'seboost_optimizer/sesop_optimizer/after_sesop/after_sesop_minibatch_loss', 
             ['lr'], scale_by_batch_size=None)

# Plotter.plot(client, ['minst_seboost_db'], query, 
#              'seboost_optimizer/full_loss', 
#              ['lr'], scale_by_batch_size=None)

plt.yscale('log')
plt.title('SESOP-SGD on MNIST Classification')
plt.xlabel('SESOP iteration')
plt.ylabel('train loss on batch')

plt.legend()
plt.grid(True)
plt.show()


databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b47160490>
experiment name: minst_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1724/events.out.tfevents.1506523917.gpu-plx03
[{u'history_decay_rate': 1.0, u'n_epochs': 100, u'seed': 913526365, u'history_size': 10, u'anchor_size': 0, u'weight_decay': 2e-05, u'normalize_history_dirs': False, u'tensorboard_dir': u'/home/shai/tensorflow/generated_data/1724', u'num_of_batches_per_sesop': 10, u'lr': 0.1, u'disable_sesop_at_all': False, u'sesop_options': {u'gtol': 1e-06, u'maxiter': 200}, u'break_sesop_batch': False, u'disable_dropout_at_all': True, u'normalize_subspace': False, u'batch_size': 100, u'anchor_offsets': [], u'iters_per_sesop': 550, u'use_grad_dir': False, u'normalize_function_during_sesop': True, u'adaptable_learning_rate': False, u'seboost_base_method': u'SGD', u'per_variable': False, u'sesop_private_dataset': False, u'VECTOR_BREAKING': False, u'disable_dropout_dur

In [411]:
accs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')

full_loss_before = []
full_loss_after = []
for i in range(len(full_loss)/2):
    full_loss_before.append(full_loss[2*i].value)
    full_loss_after.append(full_loss[2*i + 1].value)
    
    
plt.plot(full_loss_before, label='before')
plt.plot(full_loss_after, label='after')
plt.plot(full_loss_before, 'o')
plt.plot(full_loss_after, 'o')

plt.title('Full train loss before and after SESOP')
plt.xlabel('SESOP iteration')
plt.ylabel('loss')

plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b758c6710>
experiment name: minst_seboost


# The short blanket - tighten it up 2

In [408]:
# Mnist CNN with weight decay
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1765'}] }


#Plotter.accs_cache = {}
accs, _, _, configs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
print configs

Plotter.plot(client, ['minst_seboost_db'], query, 
             'seboost_optimizer/sesop_optimizer/after_sesop/after_sesop_minibatch_loss', 
             ['lr'], scale_by_batch_size=None)

# Plotter.plot(client, ['minst_seboost_db'], query, 
#              'seboost_optimizer/full_loss', 
#              ['lr'], scale_by_batch_size=None)

plt.yscale('log')
plt.title('SESOP-SGD on MNIST Classification')
plt.xlabel('SESOP iteration')
plt.ylabel('train loss on batch')

plt.legend()
plt.grid(True)
plt.show()


databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b464d0910>
experiment name: minst_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1765/events.out.tfevents.1507229823.gpu-plx03
[{u'history_decay_rate': 1.0, u'n_epochs': 100, u'seed': 913526365, u'history_size': 10, u'anchor_size': 0, u'weight_decay': 0.0001, u'normalize_history_dirs': False, u'tensorboard_dir': u'/home/shai/tensorflow/generated_data/1765', u'num_of_batches_per_sesop': 10, u'lr': 0.1, u'disable_sesop_at_all': False, u'sesop_options': {u'gtol': 1e-06, u'maxiter': 200}, u'break_sesop_batch': False, u'disable_dropout_at_all': False, u'normalize_subspace': False, u'batch_size': 100, u'anchor_offsets': [], u'iters_per_sesop': 550, u'use_grad_dir': False, u'normalize_function_during_sesop': True, u'adaptable_learning_rate': False, u'seboost_base_method': u'SGD', u'per_variable': False, u'sesop_private_dataset': False, u'VECTOR_BREAKING': False, u'disable_dropout_d

In [409]:
accs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')

full_loss_before = []
full_loss_after = []
for i in range(len(full_loss)/2):
    full_loss_before.append(full_loss[2*i].value)
    full_loss_after.append(full_loss[2*i + 1].value)
    
    
plt.plot(full_loss_before, label='before')
plt.plot(full_loss_after, label='after')
plt.plot(full_loss_before, 'o')
plt.plot(full_loss_after, 'o')

plt.title('Full train loss before and after SESOP')
plt.xlabel('SESOP iteration')
plt.ylabel('loss')

plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b37f73d10>
experiment name: minst_seboost


# The short blanket – too loosen optimization

In [395]:

# Mnist CNN without weight decay
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1728'}] }



Plotter.accs_cache = {}
accs, _, _, configs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
print configs

Plotter.plot(client, ['minst_seboost_db'], query, 
             'seboost_optimizer/sesop_optimizer/after_sesop/after_sesop_minibatch_loss', 
             ['lr', 'num_of_batches_per_sesop'], scale_by_batch_size=None)

# Plotter.plot(client, ['minst_seboost_db'], query, 
#              'seboost_optimizer/full_loss', 
#              ['lr'], scale_by_batch_size=None)

plt.yscale('log')
plt.title('SESOP-SGD on MNIST Classification')
plt.xlabel('SESOP iteration')
plt.ylabel('train loss on batch')

plt.legend()
plt.grid(True)
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b5093ee10>
experiment name: minst_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1728/events.out.tfevents.1506539478.gpu-plx03
[{u'history_decay_rate': 1.0, u'n_epochs': 100, u'seed': 913526365, u'history_size': 10, u'anchor_size': 0, u'weight_decay': 0.0, u'normalize_history_dirs': False, u'tensorboard_dir': u'/home/shai/tensorflow/generated_data/1728', u'num_of_batches_per_sesop': 10, u'lr': 0.1, u'disable_sesop_at_all': False, u'sesop_options': {u'gtol': 1e-06, u'maxiter': 200}, u'break_sesop_batch': False, u'disable_dropout_at_all': True, u'normalize_subspace': False, u'batch_size': 100, u'anchor_offsets': [], u'iters_per_sesop': 550, u'use_grad_dir': False, u'normalize_function_during_sesop': True, u'adaptable_learning_rate': False, u'seboost_base_method': u'SGD', u'per_variable': False, u'sesop_private_dataset': False, u'VECTOR_BREAKING': False, u'disable_dropout_durin

In [366]:
accs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')

full_loss_before = []
full_loss_after = []
for i in range(len(full_loss)/2):
    full_loss_before.append(full_loss[2*i].value)
    full_loss_after.append(full_loss[2*i + 1].value)
    
    
plt.plot(full_loss_before, label='before')
plt.plot(full_loss_after, label='after')
plt.plot(full_loss_before, 'o')
plt.plot(full_loss_after, 'o')

plt.title('Full train loss before and after SESOP')
plt.xlabel('SESOP iteration')
plt.ylabel('loss')

plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b35ed2b50>
experiment name: minst_seboost


#  After SESOP - adjust the learning rate (before fix)

In [407]:

# Fashion-Mnist Wide-Resnet without weight decay
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1766'}] }

accs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')
#base_optimizer/angle__u__v_

full_loss_before = []
full_loss_after = []
for i in range(len(full_loss)/2):
    full_loss_before.append(full_loss[2*i].value)
    full_loss_after.append(full_loss[2*i + 1].value)
    
    
plt.plot(full_loss_before, label='before')
plt.plot(full_loss_after, label='after')
plt.plot(full_loss_before, 'o')
plt.plot(full_loss_after, 'o')

plt.title('Full train loss before and after SESOP')
plt.xlabel('SESOP iteration')
plt.ylabel('loss')

plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b50d61ed0>
experiment name: minst_seboost


# Using SESOP to adjust the learning rate - cont 2

In [4]:
def get_angle(acc):
    dot_p = [s.value for s in \
             acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/sgd_sesop_dot_product')]
    distance_seboost = [s.value for s in \
                        acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/distance_seboost_moved')]
    distance_sesop = [s.value for s in \
                      acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/distance_sesop_moved')]

    #distance_seboost**2 = (sgd + sesop)**2 = sgd**2 + sesop**2 + 2sgd*sesop
    #sgd**2 = distance_seboost**2 - sesop**2 - 2*sgd*sesop
    
    distance_sgd = [(seboost**2 - sesop**2 - 2*d)**0.5 for seboost, sesop, d in \
                    zip(distance_seboost, distance_sesop, dot_p)]
    return [d/(sgd*sesop) for sgd, sesop, d in zip(distance_sgd, distance_sesop, dot_p)]

In [11]:
#angle
query = {'$or': [
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1942'},
   {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1969'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1992'}] }
query = {'$or': [
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1940'},
   {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1966'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1988'}] }
query = {'$or': [
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1943'},
   {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1944'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1967'}] }

accs,_,__,configs = Plotter.get_accs(client, ['minst_seboost_db'], query, 
                        ['lr', 'disable_sesop_at_all', 'tensorboard_dir'])

plt.plot(get_angle(accs[0]), label='adaptable_learning_rate = ' + str(configs[0]['adaptable_learning_rate']))

plt.plot(get_angle(accs[1]), label='adaptable_learning_rate = ' + str(configs[1]['adaptable_learning_rate']))
plt.legend()
plt.grid(True)
plt.xlabel('epoch')
plt.ylabel('Cos Angle')
plt.show()

#learning rates
lr1 = [s.value for s in \
             accs[0].Scalars('seboost_optimizer/learning_rate')]
lr2 = [s.value for s in \
             accs[1].Scalars('seboost_optimizer/learning_rate')]

plt.plot(lr1, label='adaptable_learning_rate = ' + str(configs[0]['adaptable_learning_rate']))

plt.plot(lr2, label='adaptable_learning_rate = ' + str(configs[1]['adaptable_learning_rate']))
plt.legend()
plt.grid(True)
plt.xlabel('Iteration')
plt.ylabel('Learning rate')
plt.show()


Plotter.plot(client, ['minst_seboost_db'], query, 
             'train_loss', 
             ['lr', 'disable_sesop_at_all', 'adaptable_learning_rate'], scale_by_batch_size=None)

plt.yscale('log')
plt.title('Wide Resnet on Fashion MNIST')
plt.xlabel('epoch')
plt.ylabel('train loss')

plt.legend()
plt.grid(True)
plt.show()

databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f62fecb3250>
experiment name: minst_seboost
experiment name: minst_seboost
experiment name: minst_seboost
databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f62e738b390>
experiment name: minst_seboost
experiment name: minst_seboost
experiment name: minst_seboost


In [527]:
query = {'$or': [
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1940'},
   {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1966'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/1988'}] }

# accs = Plotter.get_accs(client, ['minst_seboost_db'], query, ['lr'])
# full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')



databases = ['minst_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b4e118950>
experiment name: minst_seboost
experiment name: minst_seboost
experiment name: minst_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1940/events.out.tfevents.1507795410.gpu-plx03
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1966/events.out.tfevents.1507800441.gpu-plx03
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/1988/events.out.tfevents.1507803276.gpu-plx03


# MNIST Autoencoder optimized with SESOP-SGD

In [353]:
# query = {'$or': [
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/459'},
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/460'}] }
# query = {'$or': [
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/623'},
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/624'}] }

query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/473'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/477'}] }

Plotter.plot(client, ['minst_autoencoder_seboost_db'], query, 
             'train_loss', 
             ['lr', 'disable_sesop_at_all'], scale_by_batch_size=None)

plt.yscale('log')
plt.title('SESOP-SGD on MNIST Autoencoder')
plt.xlabel('epoch')
plt.ylabel('train loss')

plt.legend()
plt.grid(True)
plt.show()

databases = ['minst_autoencoder_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b4716bb50>
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/473/events.out.tfevents.1502037618.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/477/events.out.tfevents.1502038783.gpu-plx01


# Adjust learning rate by SESOP

In [310]:
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/477'},
    {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/478'}] }
accs = Plotter.get_accs(client, ['minst_autoencoder_seboost_db'], query, ['lr'])


databases = ['minst_autoencoder_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b362d7d50>
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/477/events.out.tfevents.1502038783.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/478/events.out.tfevents.1502039509.gpu-plx01


In [ ]:
def get_angle(acc):
    dot_p = [s.value for s in \
             acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/sgd_sesop_dot_product')]
    distance_seboost = [s.value for s in \
                        acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/distance_seboost_moved')]
    distance_sesop = [s.value for s in \
                      acc.Scalars('seboost_optimizer/sesop_optimizer/after_sesop/distance_sesop_moved')]

    #distance_seboost**2 = (sgd + sesop)**2 = sgd**2 + sesop**2 + 2sgd*sesop
    #sgd**2 = distance_seboost**2 - sesop**2 - 2*sgd*sesop
    
    distance_sgd = [(seboost**2 - sesop**2 - 2*d)**0.5 for seboost, sesop, d in \
                    zip(distance_seboost, distance_sesop, dot_p)]
    return [d/(sgd*sesop) for sgd, sesop, d in zip(distance_sgd, distance_sesop, dot_p)]

In [311]:
Plotter.plot(client, ['minst_autoencoder_seboost_db'], query, 
             'seboost_optimizer/sesop_optimizer/after_sesop/sgd_sesop_dot_product', 
             ['lr', 'adaptable_learning_rate'], scale_by_batch_size=None)

#'seboost_optimizer/sesop_optimizer/after_sesop/sgd_sesop_dot_product'

#'seboost_optimizer/full_loss'
plt.title('Adaptable learning rate affect for SESOP')
plt.xlabel('1/2 epoch')
plt.ylabel('dot product')

plt.legend()
plt.grid(True)
plt.show()


databases = ['minst_autoencoder_seboost_db']
cusor = <pymongo.cursor.Cursor object at 0x7f1b51ba8610>
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost


In [104]:
dotp = np.asarray([s.value for s in accs[0][0].Scalars('Train/Model/base_optimizer/dot__u__v_')])
u_norm = np.asarray([s.value for s in accs[0][0].Scalars('Train/Model/base_optimizer/u_norm')])
v_norm = np.asarray([s.value for s in accs[0][0].Scalars('Train/Model/base_optimizer/v_norm')])




In [105]:
domi = (u_norm**2 + 2*dotp + v_norm**2)*(u_norm**2)*(v_norm**2)
denano = dotp**2

ratio = domi/denano


/home/shai/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in divide


In [108]:
#plt.plot(ratio[2:]**0.5)
plt.plot(dotp/(u_norm*v_norm), label='dot')
#plt.plot(u_norm*v_norm)
#plt.plot(v_norm)
#plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

/home/shai/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: invalid value encountered in divide
  from ipykernel import kernelapp as app


# test accuracy

In [291]:
#{u'state_of_the_art': True, u'per_variable': False, 
#u'iters_per_adjust': 250, u'num_residual_units': 4, u'relu_leakiness': 0.1, 
#u'batch_size': 100, u'n_epochs': 100, u'base_optimizer': u'SGD', 
#u'seed': 402831889, u'lr': 0.1, u'beta2': 0.999, u'beta1': 0.9, u'rho': 0.95, u'use_bottleneck': False, 
#u'tensorboard_dir': u'/home/shai/tensorflow/generated_data/167', u'weight_decay_rate': 0.0002}


# Plotter.plot_max(client, ['cifar_ten'], {'config.state_of_the_art' : True}, 'test_accuracy', 
#                  asked_labels=['base_optimizer', 'state_of_the_art', 'iters_per_adjust'])

# query = {'$or': [{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/289'}, 
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/290'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/291'}]}
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/166'},
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/713'},
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/712'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/167'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/168'}
        #Adjust SGD
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/316'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/314'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/313'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/308'}
                #{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/290'},
                #SEBOOST
#         {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/705'},
# #                  {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/704'},
# #                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/702'},
#                  {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/712'},
        
#                  {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/711'},
#                  {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/708'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/707'}
                ]}

#Plotter.accs_cache = {}
Plotter.plot(client, ['mnist','cifar_ten', 'minst_seboost_db', 'physics_seboost_db', 'physics_alternative_db'], 
                 query, 
                 'test_loss',
                 #'train_loss',
                  asked_labels=['iters_per_adjust', 'batch_size', 'dont_run_sesop', 'lr', 'anchor_size'], 
             scale_by_batch_size=lambda b: 1.0)
plt.grid(True)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.show()

#plt.yscale('log')

experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust
experiment name: cifar_sgd_adjust


# Time to accuracy

In [270]:
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/291'},
        #Adjust SGD
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/313'},
                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/316'},
#                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/317'},
        
        #SEBOOST
                 #{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/321'},
                 {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/323'}
                #{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/290'},
                ]}

Plotter.plot_time_to_accuracy(client, ['mnist', 'minst_seboost_db'], 
                 query, 
                 'test_accuracy', 
                  asked_labels=['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch'])


plt.xlabel('test accuracy')
plt.ylabel('expected time (epoch/batch_size) to get there')
#plt.yscale('log')
#plt.yscale('log')
ax = plt.gca()
#ax.set_xticks(np.arange(0.978, 0.994, 0.0001))
plt.grid(True)
plt.legend()
plt.show()

experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_seboost


# Speedup

In [257]:
ref_query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/291'}]}

query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/291'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/296'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/308'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/313'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/314'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/316'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/317'}
                #{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/290'},
                ]}

Plotter.plot_speedup_to_accuracy(client, ['mnist'], 
                 ref_query,
                 query, 
                 'test_accuracy', 
                  asked_labels=['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch'])


plt.xlabel('test accuracy')
plt.ylabel('expected speedup to accuracy')
#plt.yscale('log')
#plt.yscale('log')
ax = plt.gca()
#ax.set_xticks(np.arange(0.978, 0.994, 0.0001))
plt.grid(True)
plt.legend()
plt.show()




experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust


# Learning rate

In [210]:
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/313'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/316'},
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/317'}
                #{'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/290'},
                ]}
Plotter.plot(client, ['mnist'], 
                 query, 
                 'base_optimizer/learning_rate', 
                 #'test_accuracy', 
                  asked_labels=['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch'], 
             scale_by_batch_size=lambda b: 1.0/b**0.5)
plt.legend()
plt.show()


experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust
experiment name: minst_sgd_adjust


In [63]:
acc = accs[0]

In [99]:

plt.show()

In [125]:
plt.xlabel('expected time (epoch/batch_size)')
plt.ylabel('test accuracy')
#plt.yscale('log')

plt.grid(True)
plt.legend()
plt.show()

In [56]:
(0.993073 - 0.992500126362)*100

0.057287363799995994

# Plot parabula

In [11]:
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/420'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/421'}
                ]}
accs = Plotter.get_accs(client, ['minst_seboost_db'], query, 
                 ['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch', 'lr'],)

# Loss

In [154]:
#326 (mnist seboost) vs 169 ()
query = {'$or': [
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/597'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/598'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/599'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/472'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/459'}
        ]}

Plotter.accs_cache = {}
Plotter.plot(client, ['minst_autoencoder_seboost_db', 'synthesised_seboost_db' ,'minst_seboost_db', 'mnist'], 
                 query, 
                 #'base_optimizer/learning_rate', 
                 'train_loss',
                 #'train_accuracy',
                 #'test_accuracy',
                  asked_labels=['disable_sesop_at_all', 'lr', 'anchor_size', 'history_size', 'sesop_private_dataset',
                               'VECTOR_BREAKING', 'normalize_function_during_sesop'], 
             scale_by_batch_size=lambda b: 1.0)

plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()


experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
experiment name: minst_autoencoder_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/459/events.out.tfevents.1502025611.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/472/events.out.tfevents.1502037604.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/597/events.out.tfevents.1502140149.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/598/events.out.tfevents.1502140158.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/599/events.out.tfevents.1502140168.gpu-plx01


In [23]:
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/420'},
        {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/421'}
                ]}
accs = Plotter.get_accs(client, ['synthesised_seboost_db'], query, 
                 ['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch', 'lr'],)

Plotter.plot(client, ['synthesised_seboost_db'], 
                 query, 
                 #'base_optimizer/learning_rate', 
                 'train_loss', 
                  asked_labels=['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch'], 
             scale_by_batch_size=lambda b: 1.0)
plt.yscale('log')
plt.grid(True)
plt.legend()
plt.show()

experiment name: synthesised_seboost
experiment name: synthesised_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/420/events.out.tfevents.1501772767.gpu-plx01
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/421/events.out.tfevents.1501772778.gpu-plx01
experiment name: synthesised_seboost
experiment name: synthesised_seboost


# Overfit debugging

In [128]:
Plotter.accs_cache = {}
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/566'}
                ]}

accs = Plotter.get_accs(client, ['synthesised_seboost_db', 'minst_seboost_db'], query, 
                 ['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch', 'lr'],)

print accs
#u'seboost_optimizer/sesop_optimizer/after_sesop/before_sesop_minibatch_loss
acc = accs[0][0]




_before = accs[0][0].Scalars('seboost_optimizer/sesop_optimizer/after_sesop/before_sesop_minibatch_loss')
after = accs[0][0].Scalars('seboost_optimizer/sesop_optimizer/after_sesop/after_sesop_minibatch_loss')
_full_loss = accs[0][0].Scalars('seboost_optimizer/full_loss')

_full_loss_before = [s for i, s in enumerate(_full_loss) if s.step%2 == 0]
full_loss_before = _full_loss_before[0:len(after)]

_full_loss_after = [s for i, s in enumerate(_full_loss) if s.step%2 == 1]
full_loss_after = _full_loss_after[0:len(after)]


#print accs[0][0].Tags()
before = _before[0:len(after)]
print len(before)
print len(after)
print len(full_loss_before)

plt.plot([s.step for s in before], [s.value for s in before], label='before')
plt.plot([s.step for s in before], [s.value for s in after], label='after')
plt.plot([s.step for s in before], [s.value for s in full_loss_before], label='full_loss_before')
plt.plot([s.step for s in before], [s.value for s in full_loss_after], label='full_loss_after')
plt.grid(True)
plt.yscale('log')
plt.legend()
plt.show()

experiment name: synthesised_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/566/events.out.tfevents.1502131710.gpu-plx01
([<tensorflow.python.summary.event_accumulator.EventAccumulator object at 0x7f5eb1610c50>], [['base_optimizer=None', 'batch_size=10', 'iters_per_adjust=None', 'weighted_batch=None', 'lr=0.09']], [10])
15
15
15


In [51]:
#326 vs 169



# Create vector drawing

In [14]:
plt.figure()
ax = plt.gca()

head_length = 0.5

#u
ax.text(3.2, 2.1, r'$u$', fontsize=24)
ax.arrow(2, 2, 2, 0, head_width=0.5, head_length=0.5, fc='g', ec='k')

#v
ax.text(5.7, 0.8, r'$v$', fontsize=24)
ax.arrow(4 + head_length, 2, 2, -2, head_width=0.5, head_length=0.5, fc='r', ec='k')

#u+v
ax.text(3.0, 0.7, r'$v+u$', fontsize=24, rotation=-25)
ax.arrow(2, 2, 4, -2, head_width=0.5, head_length=0.5, fc='b', ec='k')

#projection:
#linestyle='dashed'
#ax.text(4, 0.7, r'$\cos(2 \pi t) \exp(-t)$')
ax.text(5.0, 2.7, r'$\frac{(v+u)^Tu}{\Vert{u}\Vert}$', fontsize=24)
ax.arrow(4 + head_length, 2, 2, 0, head_width=0.5, head_length=0.5, fc='k', ec='k', linestyle='dashed')


ax.set_xlim([-1, 10])
ax.set_ylim([-1, 10])
plt.draw()
plt.show()

# GRAD NORM


In [78]:
query = {'$or': [
                {'config.tensorboard_dir' : u'/home/shai/tensorflow/generated_data/495'}
                ]}
accs = Plotter.get_accs(client, ['synthesised_seboost_db' , 'minst_autoencoder_seboost_db'], query, 
                 ['base_optimizer', 'batch_size', 'iters_per_adjust', 'weighted_batch', 'lr'],)

experiment name: synthesised_seboost
INFO:tensorflow:No path found after /home/shai/tensorflow/generated_data/495/events.out.tfevents.1502051367.gpu-plx01


In [81]:
accs[0][0].Tags()

{'audio': [],
 'compressedHistograms': [u'seboost_optimizer/sesop_optimizer/after_sesop/alpha'],
 'graph': True,
 'histograms': [u'seboost_optimizer/sesop_optimizer/after_sesop/alpha'],
 'images': [],
 'meta_graph': False,
 'run_metadata': [],
 'scalars': [u'seboost_optimizer/sesop_optimizer/after_sesop/after_sesop_minibatch_loss_normalized',
  u'seboost_optimizer/sesop_optimizer/after_sesop/sesop_grad_computations',
  u'seboost_optimizer/sesop_optimizer/after_sesop/distance_seboost_moved',
  u'seboost_optimizer/sesop_optimizer/during_sesop/grad_norm_during_sesop',
  u'seboost_optimizer/sesop_optimizer/alpha_after_sesop/a',
  u'seboost_optimizer/learning_rate',
  u'seboost_optimizer/sesop_optimizer/after_sesop/sgd_sesop_dot_product',
  u'seboost_optimizer/test_full_loss',
  u'seboost_optimizer/full_loss',
  u'seboost_optimizer/weights_norm_during_sgd',
  u'seboost_optimizer/sesop_optimizer/during_sesop/loss_during_sesop',
  u'seboost_optimizer/sesop_optimizer/after_sesop/distance_sesop

In [79]:
grad_norm = \
    accs[0][0].Scalars('seboost_optimizer/sesop_optimizer/during_sesop/grad_norm_during_sesop')

In [80]:

plt.plot([s.step for s in grad_norm], [s.value for s in grad_norm])
plt.yscale('log')
plt.grid(True)
plt.show()

In [70]:
np.linspace(24, 24, 1)

array([ 24.])